In [1]:
# testing notebook
# test for query processing time
# test for different results when different methods are used.

import sys
import os

# use abs path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

from retrieval_models.data_loaders import Index
from retrieval_models.retrieval_functions import *
from retrieval_models.reranking import Reranker
from retrieval_models.evaluation_metrics import *
import tqdm 
import os
import time
from scipy.stats import kendalltau, spearmanr


C:\Users\NAVHI\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x000001FBC5948950>>
Traceback (most recent call last):
  File "C:\Users\NAVHI\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel\ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


KeyboardInterrupt: 

In [3]:
#import all the examples
queries_examples = []
BASE_DIR = os.getcwd()
index_path = os.path.join(BASE_DIR, "data", "queries_examples.txt")
with open(index_path, 'r', encoding='utf-8', errors='replace') as file:
    queries_examples = [line.strip() for line in file if line.strip()]  # Remove empty lines

print(queries_examples[0]) #print the first one for example


TypeError: 'NoneType' object is not subscriptable in Python


In [4]:
#initialize the object index - ignore the errors is because of the preprocessing done at this stage and it ignores some terms
index = Index()

c:\Users\annie\ttds_assignment\retrieval_models\data\doc_metadata.txt


In [5]:
# Iterate over queries and print results
def run_queries(queries_examples, index):
    """
    Runs retrieval for a list of queries and prints top results.
    Also prints the total execution time.
    """
    start_time = time.time()  # Start timer

    retrieved_docs = {}
    
    for query in tqdm.tqdm(queries_examples):
        print("Query:", query)
        top_results = retrieval_function(query, index)
        print("Top results:", top_results)
        print("\n")
        retrieved_docs[query]=top_results

    end_time = time.time()  # End timer
    total_time = end_time - start_time

    print(f"Total Execution Time: {total_time:.4f} seconds")
    print(f"Average Time per Query: {total_time / len(queries_examples):.4f} seconds")

    return retrieved_docs


In [8]:
# Initialize and precompute embeddings
# will run this today to precompute and see if the results are different 
# embedding_model = EmbeddingModel(index.vocab)

In [6]:
#the basic top 10 results for each query
retrieved_docs = run_queries(queries_examples, index)

100%|██████████| 24/24 [00:00<00:00, 171.63it/s]

Query: TypeError: 'NoneType' object is not subscriptable in Python
Top results: ['26680673', '26642190', '26642203', '26691994', '26602944', '26667372', '26643796', '26684982', '26682214', '26624924']


Query: NullPointerException handling in Java
Top results: ['26624647', '26655691', '26692759', '26690439', '26675486', '26677310', '26653818', '26648853', '26624583', '26624498']


Query: ModuleNotFoundError: No module named 'requests'
Top results: ['26691973', '26691990', '26652992', '26602934', '26670457', '26679030', '26699125', '26660572', '26680255', '26697708']


Query: Segmentation fault in C++"
Top results: ['26675392', '26683088', '26671104', '26692117', '26642230', '26680273', '26683154', '26642305', '26648808', '26657098']


Query: SyntaxError: invalid syntax near 'elif'
Top results: ['26677223', '26624752', '26690465', '26624925', '26660536', '26698955', '26642234', '26676567', '26648723', '26683132']


Query: Difference between DFS and BFS algorithms
Top results: ['26682092

In [7]:
#compute_clarity_score(retrieved_docs, index)

In [ ]:
#testing to see whether the reranker returns them in different order

reranked = Reranker()
reranked_doc_list={}
for key,retrieved_docs_list in retrieved_docs.items():
    print(retrieved_docs_list)
    print("after reranking")
    reranked_doc_list[key]=reranked.rerank(retrieved_docs_list)
    print(reranked_doc_list[key])

#the documents were not matching for now - will resolve this later

['26680673', '26642190', '26642203', '26691994', '26602944', '26667372', '26643796', '26684982', '26682214', '26624924']
after reranking
⚠ Warning: No retrieved documents found in metadata.
['26680673', '26642190', '26642203', '26691994', '26602944', '26667372', '26643796', '26684982', '26682214', '26624924']
['26624647', '26655691', '26692759', '26690439', '26675486', '26677310', '26653818', '26648853', '26624583', '26624498']
after reranking
⚠ Warning: No retrieved documents found in metadata.
['26624647', '26655691', '26692759', '26690439', '26675486', '26677310', '26653818', '26648853', '26624583', '26624498']
['26691973', '26691990', '26652992', '26602934', '26670457', '26679030', '26699125', '26660572', '26680255', '26697708']
after reranking
⚠ Warning: No retrieved documents found in metadata.
['26691973', '26691990', '26652992', '26602934', '26670457', '26679030', '26699125', '26660572', '26680255', '26697708']
['26675392', '26683088', '26671104', '26692117', '26642230', '26680

In [ ]:
for key in reranked_doc_list.items():

    kendall_tau_ = kendalltau(retrieved_docs[key], reranked_doc_list[key])
    spearman_corr_ = spearmanr(retrieved_docs[key], reranked_doc_list[key])
    print('the tau value for re ranked documents')
    print(f"Kendall's Tau: {kendall_tau_}")
    print(f"Spearman’ s Rank Correlation: {spearman_corr_}")

#If τ is low but re-ranking makes sense based on metadata/LM, it may be good.
#If τ is high, re-ranking didn’t change much.
